# Advent of Code 2022: Day 14
https://adventofcode.com/2022/day/14


## Part 1
Count the number of sand units which fill up the cave, 
before the sand starts flowing out of the bottom

### Get the data into a list of strings

In [1]:
myfile = open('input.txt', 'r')
data = myfile.read()
data_list = data.split('\n')
# Remove empty value at the bottom of the list.
data_list = data_list[:-1]

### Setup helper functions

#### Function to preprocess the data

In [2]:
def preprocess_data(data, return_max_X=True, return_max_Y = True):
  
  # Setup the three objects that
  # will potentially be returned.
  output_list = []
  max_X = 0
  max_Y = 0
  
  # Go through each line
  # in the data.
  for item in data:
    
    # Setup a temporary
    # list, where the 
    # current item will go.
    temp_list = []
    
    # Split the item on ' -> '.
    split = item.split(' -> ')
    
    # Go through each item
    # that this split generated.
    for inner_item in split:
      
      # Setup a temporary list,
      # where the current inner
      # item will go.
      inner_temp_list = []
      
      # Split the inner item on ','.
      inner_split = inner_item.split(',')
      
      # Go through the two numbers
      # that this split should have
      # generated. Compare them to 
      # the current max values and
      # update if needed.
      i = 0
      for number in inner_split:
        curr = int(number)
        if i == 0:
          if curr > max_X:
            max_X = curr
          i += 1
        elif i == 1:
            if curr > max_Y:
              max_Y = curr
            i = 0
        
        # Append the current number
        # to the inner list. 
        inner_temp_list.append(curr)
      
      # Append the inner list to
      # the temporary list.
      temp_list.append(inner_temp_list)
    
    # Append the temporary list
    # to the output list.
    output_list.append(temp_list)
  

  # Return the processed data, and 
  # potentially one or two of the 
  # maximum values. 
  if return_max_X == True and return_max_Y == True:
    return output_list, max_X, max_Y
  
  elif return_max_X == True:
    return output_list, max_X
  
  elif return_max_Y == True:
    return output_list, max_Y
  
  else:
    return output_list

#### Function to get the direction and length of a line between two points

In [3]:
def get_line(point1, point2):
  
  # Get the coordinate
  # values for each point.
  x1 = point1[0]
  y1 = point1[1]
  x2 = point2[0]
  y2 = point2[1]

  # The lines cannot be diagonal,
  # which means the two points will
  # either have equal x values or 
  # equal y values. 
  if x1==x2:
    
    # Calculate the length
    # of the line. Return it, 
    # along with the direction.
    dist = y1-y2
    if dist < 0:
      return "down", dist
    else:
      return "up", dist
 
  elif y1==y2:
    
    # Calculate the length
    # of the line. Return it, 
    # along with the direction.
    dist = x1-x2
    if dist < 0:
      return "right", dist
    else:
      return "left", dist

#### Function to draw lines on a map

In [4]:
def draw_lines(map, item):
  
  # Go through each point in the item.
  for i in range(len(item)-1):
    start = item[i]
    end = item[i+1]

    # Get the direction and distance 
    # between the two points. 
    dirr, dist = get_line(start, end)

    # Starting coordinates.
    x = start[0]
    y = start[1]
    
    # Depending on the direction,
    # draw the line, using '#' to
    # as the marking symbol, for
    # the correct length.
    if dirr == "down":
      while y <= (start[1]-dist):
        map[y][x] = "#"
        y += 1

    elif dirr == "up":
      while y >=(start[1]-dist):
        map[y][x] = "#"
        y -= 1


    elif dirr == "right":
      while x <= (start[0]-dist):
        map[y][x] = "#"
        x += 1

    elif dirr == "left":
      while x >= (start[0]-dist):
        map[y][x] = "#"
        x -= 1

  return map

#### Function to create a map

In [5]:
def create_map(data, max_X, max_Y, floor = False):
  
  # Setup the map depending on the maximum
  # values provided. Pad the directions to
  # protect against going out of bounds. For
  # the y direction, just add 3 extra rows.
  # For the x direction, make the map twice
  # the length of the maximum value. This
  # is especially needed for part 2. 
  cave_map = [['.' for x in range(max_X*2)] for y in range(max_Y+3)]

  # Go through the data and
  # draw the lines on the map.
  for item in data:
    cave_map = draw_lines(cave_map, item)

  # For part 2, if floor = true, then
  # add a "floor" to the bottom of the
  # cave by manually drawing a line
  # along the bottom row.
  if floor == True:
    for col in range(len(cave_map[0])):
      cave_map[len(cave_map)-1][col] = "#"

  return cave_map

#### Function to drop sand from the roof of the cave

In [6]:
def drop_sand(map, start_point, max_Y, floor = False):
  
  # Get the coordinates
  # where the sand enters
  # the cave.
  x = start_point[0]
  y = start_point[1]

  # If the starting position
  # is already an 'o', which 
  # represents a settled unit
  # of sand, the cave is full.
  # Return the map and False.
  if map[y][x] == "o":
    return map, False

  # Otherwise, mark the starting
  # position with '+', which
  # represents a unit of sand
  # still in motion.
  map[y][x] = "+"
  while True:
    
    # If there is no floor, then
    # check if the y position of
    # the current sand unit is 
    # equal to the maximum value.
    # If it is, return the map and False.
    if floor == False:
      if y == max_Y:
        return map, False
    
    # Get the value of the map
    # in the position below
    # the current sand unit.
    below = map[y+1][x]
    
    # If the value below is '.'
    # this means there is empty
    # space below the current 
    # sand unit. If so, move
    # the sand unit down. 
    if below == ".":
      map[y+1][x] = "+"
      map[y][x] = "."
      y +=1
    
    # If the position below is not
    # empty, then get the value to
    # the bottom left (diagonally)
    # and perform the same check.
    else:
      below_left = map[y+1][x-1]
      if below_left == ".":
        map[y+1][x-1] = "+"
        map[y][x] = "."
        y += 1
        x -= 1
      
      # If the position to the bottom
      # left is also empty, then get the is not
      # value to the bottom right (diagonally)
      # and perform the same check.
      else:
        below_right = map[y+1][x+1]
        if below_right == ".":
          map[y+1][x+1] = "+"
          map[y][x] = "."
          y +=1
          x +=1
        
        # If the position to the bottom 
        # right is also empty, the sand 
        # unit can no longer keep moving.
        # Mark this by changing the value
        # from '+' to 'o'.
        else:
          map[y][x] = "o"
          break
  
  # If neither of the exit conditions
  # were met, return the map and True.
  return map, True

### Main function to count sand

In [7]:
def count_sand(data, start_point, floor = False):
  
  # Preprocess the data and get
  # the maximum values.
  data_clean, max_X, max_Y = preprocess_data(data)
  
  # Create the map of the cave.
  cave_map = create_map(data_clean, max_X, max_Y, floor)

  # Setup the sand counter.
  sand_count = 0
  
  # Keep dropping sand into the cave
  # and increment the counter until
  # the function returns False.
  while True:
    cave_map, test = drop_sand(cave_map, start_point, max_Y, floor)
    sand_count += 1
    if not test:
      return sand_count-1

In [8]:
start_point = [500,0]
count_sand(data_list, start_point)

696

## Part 2
Assuming there is a floor, count the number of sand units which fill up the cave until the cave is full

In [9]:
start_point = [500,0]
count_sand(data_list, start_point, floor=True)

23610

## Appendix
Code for printing the cave map

In [10]:
# data_clean, max_X, max_Y = preprocess_data(data_list)
# cave_map = create_map(data_clean, max_X, max_Y, floor= True)

# map_string = ""
# for i in range(len(cave_map)):
#   for j in range(len(cave_map[0])):
#     map_string += cave_map[i][j]
#     if j == 570:
#       map_string += "\n"
# print(map_string)